- 사용 데이터셋 : 구글 드라이브 files 폴더 내 파일
- 데이터셋 확인 후 지금까지 배운 내용을 기반
    - 1개 이상의 주제 만들어서 수행
    - 주제별로 별도의 파일 각각 사용
- 개별 제출

### 라이브러리 정의

In [1]:
### 기본 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 형태소 분석
from konlpy.tag import Okt

### 정규표현식(Regular Expression; regex) 
import re

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
tf.keras.utils.set_random_seed(42)

### 텍스트 길이 정규화 라이브러리
# - 텍스트의 길이가 긴경우 자르고, 길이가 작은 경우에는 채움
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 말뭉치 사전 처리를 위한 라이브러리
# - 텍스트 데이터를 숫자(인덱스번호)로 변환하는 라이브러리
from tensorflow.keras.preprocessing.text import Tokenizer

### 데이터 불러오기

In [2]:
data = pd.read_csv("./files/04_Exe_QnA_Data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Q                              11823 non-null  object
 1   A                              11823 non-null  object
 2   label(일상다반사0/이별(부정)1/사랑(긍정)2)  11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [3]:
data["Q"]

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

### 단어사전(말뭉치) 만들기

In [4]:
### 텍스트 문장 데이터를 토큰화(단어로 분리하여 숫자 인덱스화)를 위한 클래스 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["Q"] + data["A"])

In [5]:
### 전체 분리된 단어들의 갯수 확인하기
len(tokenizer.word_index)

24277

In [6]:
### 단어들의 순서 확인하기
# - 딕셔너리 타입으로 정의되어 있음(사전의 의미임)
tokenizer.word_index

{'거예요': 1,
 '수': 2,
 '너무': 3,
 '좋아하는': 4,
 '더': 5,
 '거': 6,
 '것': 7,
 '안': 8,
 '잘': 9,
 '게': 10,
 '같아요': 11,
 '있어요': 12,
 '좀': 13,
 '많이': 14,
 '있을': 15,
 '사람': 16,
 '사람이': 17,
 '마세요': 18,
 '나': 19,
 '건': 20,
 '해보세요': 21,
 '좋은': 22,
 '내가': 23,
 '내': 24,
 '썸': 25,
 '어떻게': 26,
 '왜': 27,
 '있는': 28,
 '다': 29,
 '이제': 30,
 '수도': 31,
 '좋을': 32,
 '다시': 33,
 '마음이': 34,
 '것도': 35,
 '하는': 36,
 '하고': 37,
 '또': 38,
 '시간이': 39,
 '오늘': 40,
 '정말': 41,
 '자꾸': 42,
 '될': 43,
 '다른': 44,
 '그': 45,
 '없어요': 46,
 '할': 47,
 '이별': 48,
 '바랄게요': 49,
 '돼요': 50,
 '같이': 51,
 '걸': 52,
 '헤어진지': 53,
 '좋아요': 54,
 '한': 55,
 '보세요': 56,
 '여자친구가': 57,
 '남자친구가': 58,
 '하세요': 59,
 '때': 60,
 '않아요': 61,
 '진짜': 62,
 '드세요': 63,
 '나를': 64,
 '없는': 65,
 '마음을': 66,
 '좋죠': 67,
 '먹고': 68,
 '못': 69,
 '먼저': 70,
 '바랍니다': 71,
 '일이': 72,
 '계속': 73,
 '해요': 74,
 '생각해요': 75,
 '말해보세요': 76,
 '혼자': 77,
 '뭐': 78,
 '좋겠어요': 79,
 '연락': 80,
 '있을까': 81,
 '있으면': 82,
 '몰라요': 83,
 '이렇게': 84,
 '이': 85,
 '사람은': 86,
 '그냥': 87,
 '힘든': 88,
 '사랑이': 89,
 '연애': 90,

In [7]:
### 말뭉치에 포함되지 않은 단어의 경우를 처리하기 위해
#   - 토큰화한 결과의 갯수에 1을 더하여 사용

### 훈련시 사용할 말뭉치 갯수
vocab_size = len(tokenizer.word_index) + 1
vocab_size

24278

In [8]:
### 텍스트를 말뭉치의 인덱스번호로 숫자화하는 함수 
#  - texts_to_sequences(텍스트데이터) 사용

### 질문 데이터를 인덱스화 하기
questions_sequences = tokenizer.texts_to_sequences(data["Q"])

print(len(questions_sequences))

11823


In [9]:
questions_sequences

[[8626, 8627],
 [8628, 587, 5806],
 [4375, 1399, 125],
 [4375, 788, 1399, 125],
 [8629],
 [4379],
 [4379, 524],
 [1223, 8632, 27],
 [1223, 8634, 6, 973, 256, 36, 832],
 [1223, 8636, 42],
 [8638, 95, 1638, 29],
 [425, 1419],
 [425, 3175, 1419],
 [2373, 4384, 5249],
 [8640, 8641],
 [8643, 525],
 [8644, 8645, 10718],
 [4388, 8647],
 [4388, 8649, 4391, 118],
 [3178, 3, 14, 8651],
 [3178, 4394, 1090],
 [3178, 1091, 8505],
 [238, 472, 20, 381],
 [1966, 239, 2378, 1848],
 [1966, 239, 10096],
 [1966, 239, 1639, 974],
 [1966, 108],
 [4395, 3179],
 [4396, 8657],
 [8658, 91, 150],
 [2379],
 [2379],
 [2379, 528, 974],
 [2379, 239],
 [8662],
 [1400, 3183],
 [8664, 239, 4863],
 [4398, 286],
 [8667],
 [8668, 473, 2381, 264],
 [2382, 8669],
 [2382, 2383, 832],
 [2382, 4403, 832],
 [3185, 78, 1777],
 [3185, 835],
 [8674],
 [8675, 1385],
 [885, 1967, 427, 247],
 [885, 1967],
 [4408, 4409, 68, 125],
 [4408, 4409, 4464],
 [1090, 1220],
 [1090, 2386, 7, 118],
 [1090, 3186, 108],
 [1090, 590, 6, 1220],
 [86

In [10]:
### 질문에 대한 최대값, 최소값, 중앙값 확인하기
# 52개 각각의 문장 내에 단어 갯수 확인하기
lengths = np.array([len(x) for x in questions_sequences])

np.max(lengths), np.min(lengths), np.median(lengths)

(15, 0, 3.0)

In [11]:
### 질문 데이터 데이터스케일링하기(최대 길이로 스케일링)
# - 최대 단어 길이로 스케일링하기에, 자르는값 없음. 채우는 값만 존재함
# - maxlen 속성은 15, maxlen을 생략하면 -> 전체 데이터의 단어 길이중 최대값을 사용하게됨
questions_padded = pad_sequences(questions_sequences, padding="post")
questions_padded

array([[ 8626,  8627,     0, ...,     0,     0,     0],
       [ 8628,   587,  5806, ...,     0,     0,     0],
       [ 4375,  1399,   125, ...,     0,     0,     0],
       ...,
       [24272,  1096,   151, ...,     0,     0,     0],
       [   88,    90,    22, ...,     0,     0,     0],
       [  963,     0,     0, ...,     0,     0,     0]])

In [12]:
# 답변데이터를 말뭉치사전의 인덱스로 인덱스화 하기
answers_sequences = tokenizer.texts_to_sequences(data["A"])

### 답변에 대한 최대값, 최소값, 중앙값 확인하기
# 52개 각각의 문장 내에 단어 갯수 확인하기
lengths = np.array([len(x) for x in answers_sequences])
np.max(lengths), np.min(lengths), np.median(lengths)

(21, 0, 3.0)

In [13]:
### 답변 데이터 데이터스케일링하기(최대 길이로 스케일링)
answers_padded = pad_sequences(answers_sequences, padding="post")
answers_padded

array([[ 1963,    38,  2371, ...,     0,     0,     0],
       [ 1033,  2372,     0, ...,     0,     0,     0],
       [ 3508,   380,    67, ...,     0,     0,     0],
       ...,
       [24273,     0,     0, ...,     0,     0,     0],
       [    9,  1585,     2, ...,     0,     0,     0],
       [20196,  1887,   104, ...,     0,     0,     0]])

In [14]:
### 독립변수 = 질문 데이터
#   종속변수 = 답변 데이터
# 변수명 : questions_train, answers_train, questions_val, answers_val
questions_train, questions_val, answers_train, answers_val = train_test_split(
    questions_padded, answers_padded, test_size=0.2, random_state=42
)
print(questions_train.shape, answers_train.shape)
print(questions_val.shape, answers_val.shape)

(9458, 15) (9458, 21)
(2365, 15) (2365, 21)


In [20]:

### 모델 생성하기
model = keras.Sequential()
model

### 입력계층 추가하기(임베딩 계층 추가하기)
#    - input_dim : 말뭉치갯수
#    - 출력갯수 : 64개
#    - input_length : 질문의 특성 갯수
model.add(
    keras.layers.Embedding(input_dim=vocab_size,
                           output_dim=64,
                           input_length=questions_train.shape[1])
)

### 은닉계층 추가 : GRU, 출력:128, 활성화함수:relu
# - 질문을 담당하는 계층
model.add(
    keras.layers.GRU(units=128, activation="relu")
)

### 드롭아웃 추가
model.add(keras.layers.Dropout(0.3))

### 질문을 담당하는 계층에서 넘어오는 결과는 6개의 질문 특성을 사용함
# - 답변차원의 특성 7개로 변경한 후 답변을 담당하는 계층으로 넘겨주기
model.add(
    keras.layers.RepeatVector(answers_train.shape[1])
)

### 은닉계층 : GRU 계층(질문 결과를 받아서 답변과의 일치성 훈련 시키기)
# - return_sequences=True : 훈련결과(단어)를 다음 계층으로 넘겨주기
#                         : 다음계층에서 처리결과 단어들을 받아서 연속에서 훈련 진행
model.add(
    keras.layers.GRU(units=128, activation="relu", return_sequences=True)
)

### 단어조합 계층과 출력계층 정의하기
# TimeDistributed 
#  - 전체 문장을 기준으로 위 계층에서 전달받은 단어들의 이전/다음 인덱스의 연결(문맥 연결)을 관리하는 계층
#  - 다음에 올 단어들이 있는지 체크
#  - 분류의 개념보다, 예측(회귀-시간적 흐름-단어의 문맥)의 개념을 담고있는 계층임
#  - 출력계층을 감싸서 사용
# ** 처리 순서 : 출력계층의 말뭉치 결과를 TimeDistributed 계층에서 확률이 높은 단어들을 조합하여 반환
model.add(
    keras.layers.TimeDistributed(
        keras.layers.Dense(units=vocab_size, activation="softmax")
    )
)

model.summary()

### 모델 설정하기
# - rmsprop 사용, 학습율 기존값사용, 정확도 출력
model.compile(
    optimizer="RMSprop",
    loss="sparse_categorical_crossentropy",
    metrics="accuracy"
)

### 콜백함수 정의하기
# - 파일명 : best_RNN_chatbot.h5
save_file = "./model/best_RNN_chatbot.h5"
cp_cb = keras.callbacks.ModelCheckpoint(save_file, save_best_only=True)
es_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

### 훈련시키기
# - 훈련횟수:100회, 배치사이즈:64
history = model.fit(
    questions_padded, answers_padded, epochs=500, batch_size=64 
    # , validation_data=(questions_val, answers_val)
    # , callbacks=[cp_cb, es_cb]
)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 15, 64)            1553792   
                                                                 
 gru_4 (GRU)                 (None, 128)               74496     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 repeat_vector_5 (RepeatVect  (None, 21, 128)          0         
 or)                                                             
                                                                 
 gru_5 (GRU)                 (None, 21, 128)           99072     
                                                                 
 time_distributed_4 (TimeDis  (None, 21, 24278)        3131862   
 tributed)                                            

KeyboardInterrupt: 